# Data Preparation for AWS SageMaker Linear Learner

This notebook prepares any CSV dataset for use with AWS SageMaker's Linear Learner algorithm.

**You only need to change two things:**
1. The filename of your dataset
2. The name of your target column (what you want to predict)

Run each cell in order from top to bottom. At the end, you will have a cleaned CSV file ready for SageMaker.

**Important:** If you need to start over, use Kernel → Restart & Clear Output before re-running.

---

## Choosing a Good Dataset

### Size Requirements
- **Minimum:** 300 rows (500+ recommended)
- **Columns:** At least 4 feature columns plus your target

### Data Quality
- Prefer datasets with mostly numeric columns
- Avoid text-heavy datasets (reviews, descriptions)
- Look for datasets described as "clean"

### Thresholds This Notebook Uses

| Threshold | Action |
|-----------|--------|
| >30% missing | Column removed |
| >50 text categories | Column removed |
| ≤25 text categories | One-hot encoded (dummy variables) |
| 26-50 text categories | Frequency encoded |
| >20 target categories | Error |
| ≤10 unique target values | Treated as classification |

### Terminology

| Term | Meaning |
|------|---------|
| Features | Input columns used for prediction |
| Target | The column you want to predict |
| Regression | Predicting a number |
| Classification | Predicting a category |
| One-hot encoding | Converting categories to yes/no columns (also called dummy variables) |

---
## STEP 1: Configuration

**EDIT THE TWO VARIABLES BELOW**, then run this cell.

In [1]:
# ============================================================
# CHANGE THESE TWO VALUES
# ============================================================

FILE_NAME = "your_dataset.csv"       # Your CSV file name
TARGET_COLUMN = "your_target"        # The column you want to predict

# ============================================================
# DO NOT CHANGE ANYTHING BELOW THIS LINE
# ============================================================

# Thresholds for automated cleaning
MISSING_THRESHOLD = 0.30          # Drop columns with more than 30% missing
HIGH_CARDINALITY_THRESHOLD = 50   # Drop text columns with more than 50 unique values
ONEHOT_THRESHOLD = 25             # One-hot encode if 25 or fewer unique values
MAX_TEXT_CATEGORIES = 20          # Maximum categories allowed for text target columns

# Validate thresholds (in case someone edits them)
assert 0 < MISSING_THRESHOLD <= 1, "MISSING_THRESHOLD must be between 0 and 1"
assert HIGH_CARDINALITY_THRESHOLD > 0, "HIGH_CARDINALITY_THRESHOLD must be positive"
assert ONEHOT_THRESHOLD > 0, "ONEHOT_THRESHOLD must be positive"

# Strip any accidental whitespace from user input
FILE_NAME = FILE_NAME.strip()
TARGET_COLUMN = TARGET_COLUMN.strip()

print(f"Configuration set:")
print(f"  File: {FILE_NAME}")
print(f"  Target: {TARGET_COLUMN}")

Configuration set:
  File: AmesHousing.csv
  Target: SalePrice


In [2]:
# ============================================================
# TRACKING VARIABLES (initialized for safety)
# ============================================================
# These track what changes are made during processing.
# Do not modify - they are set automatically by later steps.

PROBLEM_TYPE = None
ORIGINAL_CLASS_MAPPING = None
RAW_ROWS = 0
RAW_COLS = 0
COLS_DROPPED_USELESS = 0
COLS_DROPPED_MISSING = 0
COLS_DROPPED_CARDINALITY = 0
COLS_ENCODED = 0
ROWS_DROPPED_MISSING = 0
ROWS_DROPPED_DUPLICATES = 0

print("Tracking variables initialized.")

Tracking variables initialized.


---
## STEP 2: Load Required Libraries

In [3]:
import os
import re
import csv
import json
import numpy as np
import pandas as pd
import warnings

# Only suppress version-related warnings, keep data warnings visible
warnings.filterwarnings('ignore', category=FutureWarning)

print("Libraries loaded successfully.")
print(f"  pandas version: {pd.__version__}")
print(f"  numpy version: {np.__version__}")

Libraries loaded successfully.
  pandas version: 2.3.3
  numpy version: 1.26.4


---
## STEP 3: Load and Inspect Your Data

In [4]:
# Helper function to detect delimiter (ENHANCED: examines multiple lines for robustness)
def detect_delimiter(filename):
    """Auto-detect CSV delimiter by examining multiple lines for consistency."""
    # Read first 5 lines for more robust detection
    lines = []
    used_encoding = 'utf-8-sig'
    for encoding in ['utf-8-sig', 'utf-8', 'latin1']:
        try:
            with open(filename, 'r', encoding=encoding) as f:
                lines = [f.readline() for _ in range(5)]
            used_encoding = encoding
            break
        except UnicodeDecodeError:
            continue
    
    if not lines or not any(line.strip() for line in lines):
        return ',', 'comma'
    
    # Score each delimiter by consistency across lines
    best_delimiter = ','
    best_score = 0
    
    for delimiter in ['\t', ';', ',']:
        try:
            counts = []
            for line in lines:
                if line.strip():
                    reader = csv.reader([line], delimiter=delimiter)
                    counts.append(len(next(reader)))
            
            if counts and len(set(counts)) == 1 and counts[0] > 1:
                # Consistent field count across lines - good sign
                if counts[0] > best_score:
                    best_score = counts[0]
                    best_delimiter = delimiter
        except:
            continue
    
    delim_names = {'\t': 'tab', ';': 'semicolon', ',': 'comma'}
    return best_delimiter, delim_names.get(best_delimiter, 'comma')

# Detect delimiter first
try:
    delimiter, delim_name = detect_delimiter(FILE_NAME)
except FileNotFoundError:
    print(f"ERROR: Could not find file '{FILE_NAME}'")
    print(f"Make sure the file is in the same folder as this notebook.")
    raise

# Load the data (FIX: use utf-8-sig to handle BOM)
try:
    df = pd.read_csv(FILE_NAME, sep=delimiter, encoding='utf-8-sig')
    if delim_name != 'comma':
        print(f"Detected {delim_name}-separated file.")
except UnicodeDecodeError:
    # Try alternative encodings
    for encoding in ['latin1', 'iso-8859-1', 'cp1252']:
        try:
            df = pd.read_csv(FILE_NAME, sep=delimiter, encoding=encoding)
            print(f"Loaded with {encoding} encoding.")
            break
        except:
            continue
    else:
        print(f"ERROR: Could not read file. Try saving it as UTF-8.")
        raise

# Strip whitespace from column names (common source of errors)
df.columns = df.columns.str.strip()

# Check if file is empty
if len(df) == 0:
    print(f"ERROR: The file '{FILE_NAME}' contains no data rows.")
    print("Please ensure your CSV file contains data, not just headers.")
    raise ValueError("CSV file is empty (no data rows).")

# Check if target column exists (with auto-correction for case mismatches)
if TARGET_COLUMN not in df.columns:
    # Check for close matches (case differences, extra spaces)
    close_matches = [c for c in df.columns if c.lower().strip() == TARGET_COLUMN.lower().strip()]
    if len(close_matches) == 1:
        # Auto-correct if exactly one case-insensitive match
        print(f"Note: Using '{close_matches[0]}' (case-corrected from '{TARGET_COLUMN}')")
        TARGET_COLUMN = close_matches[0]
    else:
        print(f"ERROR: Could not find column '{TARGET_COLUMN}'")
        print(f"")
        if close_matches:
            print(f"Did you mean one of these? {close_matches}")
            print(f"")
        print(f"Available columns are:")
        for col in df.columns:
            print(f"  - {col}")
        raise ValueError(f"Target column '{TARGET_COLUMN}' not found.")

# Save initial dimensions for final report
RAW_ROWS = len(df)
RAW_COLS = len(df.columns)

# Warn about very small datasets
if RAW_ROWS < 300:
    print(f"WARNING: The file contains only {RAW_ROWS} rows.")
    print("Machine learning typically requires hundreds or thousands of rows.")
    print("")

# Check for potential date/time columns
date_patterns = ['date', 'time', 'year', 'month', 'day', 'week', 
                 'timestamp', 'created', 'updated', 'modified', 'period']
potential_date_cols = [col for col in df.columns 
                       if any(pattern in col.lower() for pattern in date_patterns)]

# Also check for columns that look like dates based on content
for col in df.select_dtypes(include=['object']).columns:
    if col not in potential_date_cols and col != TARGET_COLUMN:
        # Sample a few values to check for date-like patterns
        sample = df[col].dropna().head(5).astype(str)
        # Common date patterns: YYYY-MM-DD, MM/DD/YYYY, etc.
        date_like = sample.str.match(r'^\d{1,4}[-/]\d{1,2}[-/]\d{1,4}').any()
        if date_like:
            potential_date_cols.append(col)

if potential_date_cols:
    print("-" * 60)
    print("NOTE: Potential date/time columns detected:")
    for col in potential_date_cols:
        print(f"  - {col}")
    print()
    print("Date columns are currently processed as text and may be removed.")
    print("For better results, consider converting dates to separate columns")
    print("(Year, Month, Day of Week) in Excel before uploading.")
    print("-" * 60)
    print()

print("=" * 60)
print("DATA LOADED SUCCESSFULLY")
print("=" * 60)
print(f"Rows: {RAW_ROWS:,}")
print(f"Columns: {RAW_COLS}")
print(f"Target column: {TARGET_COLUMN}")
print(f"")
print("First few rows:")
df.head()

------------------------------------------------------------
NOTE: Potential date/time columns detected:
  - Year Built
  - Year Remod/Add

Date columns are currently processed as text and may be removed.
For better results, consider converting dates to separate columns
(Year, Month, Day of Week) in Excel before uploading.
------------------------------------------------------------

DATA LOADED SUCCESSFULLY
Rows: 2,930
Columns: 82
Target column: SalePrice

First few rows:


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


---
### Data Overview

Review the columns loaded above. Consider:
- What type of information does this dataset contain?
- What patterns might exist in this data?
- Is the target column appropriate for your prediction goal?

---
## STEP 4: Detect Problem Type

The notebook determines if this is regression or classification.

### Problem Types

**Regression** - predicting a number:
- Price, revenue, count, percentage

**Classification** - predicting a category:
- Yes/No, Low/Medium/High, Category A/B/C

### Detection Rules

1. Text target with ≤20 categories → Classification
2. Numeric target with ≤10 unique values → Classification  
3. Numeric target with >10 unique values → Regression

In [5]:
# Analyze the target column to determine problem type
target_values = df[TARGET_COLUMN].dropna()
n_unique = target_values.nunique()
target_dtype = target_values.dtype

print("=" * 60)
print("ANALYZING TARGET COLUMN")
print("=" * 60)
print(f"Column: {TARGET_COLUMN}")
print(f"Unique values: {n_unique}")
print(f"Data type: {target_dtype}")
print("")

# Check for degenerate case first (must have at least 2 unique values)
if n_unique < 2:
    print(f"ERROR: Target column '{TARGET_COLUMN}' has only {n_unique} unique value(s).")
    print("A model cannot learn to predict when there is no variation in the target.")
    print("Please check your dataset or choose a different target column.")
    raise ValueError(f"Target column must have at least 2 unique values.")

# Determine problem type (ENHANCED: better handling of float vs int targets)
if target_dtype == 'object':
    # Text target - must be classification
    if n_unique > MAX_TEXT_CATEGORIES:
        print(f"ERROR: Target column has {n_unique} text categories.")
        print(f"This is too many for classification (maximum {MAX_TEXT_CATEGORIES}).")
        print("Consider:")
        print("  - Grouping categories into fewer buckets")
        print("  - Using a different target column")
        print("  - Using a different dataset")
        raise ValueError(f"Too many categories ({n_unique}) in text target column.")
    elif n_unique == 2:
        PROBLEM_TYPE = 'binary_classification'
        print("Detected: BINARY CLASSIFICATION (two text categories)")
    else:
        PROBLEM_TYPE = 'multiclass_classification'
        print(f"Detected: MULTICLASS CLASSIFICATION ({n_unique} text categories)")
elif pd.api.types.is_integer_dtype(target_values) and n_unique <= 10:
    # Integer with few unique values - treat as classification
    if n_unique == 2:
        PROBLEM_TYPE = 'binary_classification'
        print("Detected: BINARY CLASSIFICATION (two numeric categories)")
    else:
        PROBLEM_TYPE = 'multiclass_classification'
        print(f"Detected: MULTICLASS CLASSIFICATION ({n_unique} numeric categories)")
elif pd.api.types.is_float_dtype(target_values) and n_unique == 2:
    # Float with only 2 unique values - check if 0.0 and 1.0
    unique_vals = sorted(target_values.unique())
    if unique_vals == [0.0, 1.0]:
        PROBLEM_TYPE = 'binary_classification'
        print("Detected: BINARY CLASSIFICATION (0.0 and 1.0 values)")
    else:
        # ENHANCED: Provide guidance for ambiguous 2-value float targets
        PROBLEM_TYPE = 'regression'
        print("Detected: REGRESSION (predicting a number)")
        print(f"")
        print(f"Note: Your target has only 2 unique float values: {unique_vals}")
        print("If this should be classification (predicting categories), convert your")
        print("target column to integers or text first, then re-run the notebook.")
elif pd.api.types.is_float_dtype(target_values) and n_unique <= 10:
    # ENHANCED: Provide note for float targets with few unique values
    PROBLEM_TYPE = 'regression'
    print("Detected: REGRESSION (predicting a number)")
    print(f"")
    print(f"Note: Your target has only {n_unique} unique values.")
    print("If these represent categories (like ratings 1-5), consider converting")
    print("the target to integers for classification instead of regression.")
else:
    PROBLEM_TYPE = 'regression'
    print("Detected: REGRESSION (predicting a number)")

# For classification, encode target as integers
# (FIX: store original mapping for potential re-encoding later)
ORIGINAL_CLASS_MAPPING = None
if PROBLEM_TYPE in ['binary_classification', 'multiclass_classification']:
    # First report any missing targets
    missing_target_count = df[TARGET_COLUMN].isna().sum()
    if missing_target_count > 0:
        print(f"")
        print(f"Note: {missing_target_count} rows have missing target values (will be removed later).")
    
    print("")
    print("Converting categories to numbers:")
    
    # FIX: Handle mixed types by converting to string first
    try:
        unique_classes = sorted(df[TARGET_COLUMN].dropna().unique())
    except TypeError:
        # Mixed types - convert all to string first
        print("Note: Target column has mixed types. Converting all to text.")
        df[TARGET_COLUMN] = df[TARGET_COLUMN].astype(str)
        unique_classes = sorted(df[TARGET_COLUMN].dropna().unique())
    
    ORIGINAL_CLASS_MAPPING = {val: idx for idx, val in enumerate(unique_classes)}
    df[TARGET_COLUMN] = df[TARGET_COLUMN].map(ORIGINAL_CLASS_MAPPING)
    for original, encoded in ORIGINAL_CLASS_MAPPING.items():
        print(f"  {original} -> {encoded}")

print("")
print("-" * 60)
print("Does this match your prediction from above?")
print("If not, review the 'After Running the Next Cell' section.")
print("-" * 60)

ANALYZING TARGET COLUMN
Column: SalePrice
Unique values: 1032
Data type: int64

Detected: REGRESSION (predicting a number)

------------------------------------------------------------
Does this match your prediction from above?
If not, review the 'After Running the Next Cell' section.
------------------------------------------------------------


---
## STEP 5: Remove Useless Columns

This step removes columns that would confuse the model:
- ID columns (just labels, not useful for prediction)
- Constant columns (same value in every row, provides no information)

### Why These Columns Get Removed

**ID columns** (like customer_id, order_number): These are just labels that identify each row. They do not contain information that helps predict outcomes. Think of them like Social Security Numbers - unique to each person but not predictive of behavior.

**Constant columns**: If every row has the same value, that column cannot help distinguish between different outcomes. A column where everyone has the same value provides zero predictive power.

In [6]:
print("=" * 60)
print("REMOVING USELESS COLUMNS")
print("=" * 60)

cols_to_drop = []
drop_reasons = []

# ID-like column name patterns (whole word matches only)
ID_EXACT_NAMES = {'id', 'index', 'rownum', 'row_num', 'record', 'record_id', 'row_id', 
                  'idx', 'pk', 'key', 'guid', 'uuid'}
ID_SUFFIXES = {'_id', '_index', '_idx', '_pk', '_key'}
ID_PREFIXES = {'id_', 'index_', 'idx_'}

for col in df.columns:
    if col == TARGET_COLUMN:
        continue
    
    col_lower = col.lower()
    
    # Check for ID-like column names (whole word match, not substring)
    is_id_name = (
        col_lower in ID_EXACT_NAMES or
        any(col_lower.endswith(suffix) for suffix in ID_SUFFIXES) or
        any(col_lower.startswith(prefix) for prefix in ID_PREFIXES)
    )
    
    # Check if values are unique (strong ID indicator)
    is_unique = df[col].nunique() == len(df)
    
    # Check if values are sequential integers (very strong ID indicator)
    is_sequential = False
    if pd.api.types.is_numeric_dtype(df[col]):
        col_sorted = df[col].dropna().sort_values().reset_index(drop=True)
        if len(col_sorted) > 1:
            diffs = col_sorted.diff().dropna()
            if len(diffs) > 0 and ((diffs == 1).all() or (diffs == -1).all()):
                is_sequential = True
    
    # ENHANCED: Check if column looks like categorical data despite unique values
    # This prevents removing columns like 'category_id' that contain meaningful categories
    is_likely_categorical = False
    if df[col].dtype == 'object' and is_unique:
        # If values are short strings (< 30 chars avg), they might be categories not IDs
        avg_len = df[col].astype(str).str.len().mean()
        # Also check if values look like UUIDs or long codes
        sample_vals = df[col].dropna().head(5).astype(str)
        looks_like_uuid = any(len(str(v)) > 30 or '-' in str(v) for v in sample_vals)
        if avg_len < 25 and not looks_like_uuid:
            is_likely_categorical = True
    
    # Only flag as ID if: (name looks like ID AND unique AND not categorical) OR (sequential integers)
    if (is_id_name and is_unique and not is_likely_categorical) or (is_sequential and is_unique):
        cols_to_drop.append(col)
        if is_sequential:
            drop_reasons.append(f"'{col}' - sequential ID column")
        else:
            drop_reasons.append(f"'{col}' - ID column (just a label)")
        continue
    
    # Check for constant columns (all same value)
    if df[col].nunique() <= 1:  # <= 1 handles all-NaN case too
        cols_to_drop.append(col)
        drop_reasons.append(f"'{col}' - constant (all values identical)")

# Execute drops
if cols_to_drop:
    df = df.drop(columns=cols_to_drop)
    print(f"Removed {len(cols_to_drop)} column(s):")
    for reason in drop_reasons:
        print(f"  - {reason}")
else:
    print("No useless columns found.")

COLS_DROPPED_USELESS = len(cols_to_drop)

# Check if any features remain
remaining_features = [c for c in df.columns if c != TARGET_COLUMN]
if len(remaining_features) == 0:
    print("")
    print("!" * 60)
    print("ERROR: NO FEATURES REMAINING")
    print("!" * 60)
    print("All columns except the target were removed.")
    print("This may indicate:")
    print("  - All columns look like ID columns")
    print("  - All columns have identical values")
    print("Please choose a different dataset.")
    raise ValueError("No feature columns remaining.")

print(f"")
print(f"Data shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

REMOVING USELESS COLUMNS
Removed 1 column(s):
  - 'Order' - sequential ID column

Data shape: 2,930 rows x 81 columns


---
## STEP 6: Handle Missing Values

Missing values cause problems for machine learning. This step:
- Removes rows where the target is missing (we cannot learn from these)
- Removes columns with too many missing values
- Fills remaining gaps with reasonable estimates

In [7]:
print("=" * 60)
print("HANDLING MISSING VALUES")
print("=" * 60)

rows_before = len(df)
cols_before = len(df.columns)

# 1. Drop rows with missing target
missing_target = df[TARGET_COLUMN].isna().sum()
if missing_target > 0:
    df = df.dropna(subset=[TARGET_COLUMN])
    print(f"Removed {missing_target} rows with missing target values.")

# Check if we have any data left after removing missing targets
if len(df) == 0:
    print("")
    print("!" * 60)
    print("ERROR: ALL ROWS REMOVED")
    print("!" * 60)
    print("Every row had a missing target value.")
    print("Please check your target column or choose a different dataset.")
    raise ValueError("No rows remaining after removing missing targets.")

# 2. Drop columns with too many missing values
cols_dropped_missing = []
for col in df.columns:
    if col == TARGET_COLUMN:
        continue
    missing_pct = df[col].isna().sum() / len(df)
    if missing_pct > MISSING_THRESHOLD:
        cols_dropped_missing.append(col)

if cols_dropped_missing:
    df = df.drop(columns=cols_dropped_missing)
    print(f"Removed {len(cols_dropped_missing)} column(s) with more than {MISSING_THRESHOLD*100:.0f}% missing:")
    for col in cols_dropped_missing:
        print(f"  - {col}")

# FIX: Convert boolean columns to integer before processing
bool_cols = df.select_dtypes(include=['bool']).columns.tolist()
if bool_cols:
    for col in bool_cols:
        df[col] = df[col].astype(int)
    print(f"Converted {len(bool_cols)} boolean column(s) to integers.")

# ENHANCED: Replace infinite values with NaN before median calculation
# This prevents median from being inf and ensures proper handling
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
inf_count = 0
for col in numeric_cols:
    col_inf = np.isinf(df[col]).sum()
    if col_inf > 0:
        inf_count += col_inf
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)

if inf_count > 0:
    print(f"Replaced {inf_count} infinite value(s) with missing (will be filled with median).")

# 3. Fill remaining missing values
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Fill numeric with median (handle all-NaN columns)
cols_dropped_all_nan = []
for col in numeric_cols:
    if col != TARGET_COLUMN and df[col].isna().sum() > 0:
        median_val = df[col].median()
        if pd.isna(median_val):
            # Column is entirely NaN - drop it
            cols_dropped_all_nan.append(col)
        else:
            df.loc[:, col] = df[col].fillna(median_val)

if cols_dropped_all_nan:
    df = df.drop(columns=cols_dropped_all_nan)
    print(f"Removed {len(cols_dropped_all_nan)} column(s) with all missing values:")
    for col in cols_dropped_all_nan:
        print(f"  - {col}")

# Fill categorical with 'Missing'
for col in categorical_cols:
    if df[col].isna().sum() > 0:
        df.loc[:, col] = df[col].fillna('Missing')

rows_after = len(df)
cols_after = len(df.columns)

print("")
print(f"Rows: {rows_before:,} -> {rows_after:,} ({rows_before - rows_after:,} removed)")
print(f"Columns: {cols_before} -> {cols_after} ({cols_before - cols_after} removed)")

# Check if we still have data
if len(df) < 10:
    print("")
    print("!" * 60)
    print("ERROR: TOO FEW ROWS REMAINING")
    print("!" * 60)
    print(f"Only {len(df)} rows remain after cleaning.")
    print("This dataset has too many missing values to use.")
    print("Please choose a different dataset.")
    raise ValueError("Insufficient data remaining.")

# Track for final summary
ROWS_DROPPED_MISSING = rows_before - rows_after
COLS_DROPPED_MISSING = len(cols_dropped_missing) + len(cols_dropped_all_nan)

print(f"")
print(f"Data shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

HANDLING MISSING VALUES
Removed 6 column(s) with more than 30% missing:
  - Alley
  - Mas Vnr Type
  - Fireplace Qu
  - Pool QC
  - Fence
  - Misc Feature

Rows: 2,930 -> 2,930 (0 removed)
Columns: 81 -> 75 (6 removed)

Data shape: 2,930 rows x 75 columns


---
## STEP 7: Encode Categorical Variables

Machine learning requires all data to be numeric. This step converts text columns to numbers:
- Columns with few categories become multiple yes/no columns (one-hot encoding, also known as **dummy variables**)
- Columns with many categories are converted based on frequency
- Columns with too many categories are removed

In [8]:
print("=" * 60)
print("ENCODING NON-NUMERIC COLUMNS")
print("=" * 60)

# FIRST: Attempt to convert "dirty" numeric columns (currency, commas, percentages)
# Business data often has values like "$1,200.00" or "1,000" stored as text
object_cols = df.select_dtypes(include=['object']).columns.tolist()
cols_converted_to_numeric = []

for col in object_cols:
    if col == TARGET_COLUMN:
        continue
    
    # Try to clean and convert to numeric
    # Remove common formatting: $, commas, %, leading/trailing spaces
    cleaned = df[col].astype(str).str.strip()
    cleaned = cleaned.str.replace(r'[$,]', '', regex=True)
    cleaned = cleaned.str.replace(r'%$', '', regex=True)
    
    # Attempt conversion
    numeric_version = pd.to_numeric(cleaned, errors='coerce')
    
    # Only convert if 90%+ of non-null values successfully converted
    non_null_count = df[col].notna().sum()
    if non_null_count > 0:
        success_rate = numeric_version.notna().sum() / non_null_count
        if success_rate >= 0.90:
            df[col] = numeric_version
            cols_converted_to_numeric.append(col)

if cols_converted_to_numeric:
    print(f"Converted {len(cols_converted_to_numeric)} text column(s) to numbers:")
    for col in cols_converted_to_numeric:
        print(f"  - {col} (detected currency/number formatting)")
    print()

# NOW proceed with categorical encoding for remaining object columns
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

if not categorical_cols:
    print("No non-numeric columns to encode.")
    COLS_ENCODED = 0
    COLS_DROPPED_CARDINALITY = 0
else:
    cols_encoded = 0
    cols_dropped_cardinality = []
    
    for col in categorical_cols:
        n_unique = df[col].nunique()
        
        if n_unique <= ONEHOT_THRESHOLD:
            # One-hot encode (creates separate columns for each category)
            # drop_first=True removes one category to avoid multicollinearity
            dummies = pd.get_dummies(df[col], prefix=col, drop_first=True, dtype=int)
            
            # Check for naming conflicts with existing columns
            conflicting_cols = set(dummies.columns) & set(df.columns) - {col}
            if conflicting_cols:
                # Rename conflicting columns to avoid silent overwrites
                dummies = dummies.rename(columns={c: f"{c}_enc" for c in conflicting_cols})
                print(f"  Note: Renamed {len(conflicting_cols)} column(s) to avoid naming conflicts.")
            
            df = pd.concat([df.drop(columns=[col]), dummies], axis=1)
            print(f"  '{col}' ({n_unique} categories) -> {len(dummies.columns)} dummy column(s)")
            cols_encoded += 1
            
        elif n_unique <= HIGH_CARDINALITY_THRESHOLD:
            # Frequency encode (replace with how common each value is)
            # Note: Frequency encoding replaces each category with its occurrence rate.
            # This works reasonably for Linear Learner but is not ideal - the model
            # assumes a linear relationship between frequency and the target, which
            # may not exist. For better results, consider reducing categories or
            # using a dataset with fewer high-cardinality text columns.
            freq_map = df[col].value_counts(normalize=True).to_dict()
            df[col] = df[col].map(freq_map)
            
            # FIX: Verify no NaN was introduced
            if df[col].isna().any():
                print(f"  Warning: Some values in '{col}' could not be frequency encoded. Filling with 0.")
                df.loc[:, col] = df[col].fillna(0)
            
            print(f"  '{col}' ({n_unique} categories) -> frequency values")
            cols_encoded += 1
            
        else:
            # Too many unique values - drop the column
            df = df.drop(columns=[col])
            cols_dropped_cardinality.append(col)
            print(f"  '{col}' ({n_unique} categories) -> REMOVED (too many categories)")
    
    print("")
    print(f"Encoded {cols_encoded} column(s).")
    if cols_dropped_cardinality:
        print(f"Removed {len(cols_dropped_cardinality)} column(s) with too many categories.")
    
    COLS_ENCODED = cols_encoded
    COLS_DROPPED_CARDINALITY = len(cols_dropped_cardinality)

# FIX: Check if any features remain after encoding
remaining_features = [c for c in df.columns if c != TARGET_COLUMN]
if len(remaining_features) == 0:
    print("")
    print("!" * 60)
    print("ERROR: NO FEATURES REMAINING")
    print("!" * 60)
    print("All feature columns were removed due to too many categories.")
    print("Consider:")
    print("  - Using a dataset with fewer text columns")
    print("  - Using a dataset with more rows")
    print("  - Increasing HIGH_CARDINALITY_THRESHOLD (advanced)")
    raise ValueError("No feature columns remaining after encoding.")

print(f"")
print(f"Data shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

ENCODING NON-NUMERIC COLUMNS
  'MS Zoning' (7 categories) -> 6 dummy column(s)
  'Street' (2 categories) -> 1 dummy column(s)
  'Lot Shape' (4 categories) -> 3 dummy column(s)
  'Land Contour' (4 categories) -> 3 dummy column(s)
  'Utilities' (3 categories) -> 2 dummy column(s)
  'Lot Config' (5 categories) -> 4 dummy column(s)
  'Land Slope' (3 categories) -> 2 dummy column(s)
  'Neighborhood' (28 categories) -> frequency values
  'Condition 1' (9 categories) -> 8 dummy column(s)
  'Condition 2' (8 categories) -> 7 dummy column(s)
  'Bldg Type' (5 categories) -> 4 dummy column(s)
  'House Style' (8 categories) -> 7 dummy column(s)
  'Roof Style' (6 categories) -> 5 dummy column(s)
  'Roof Matl' (8 categories) -> 7 dummy column(s)
  'Exterior 1st' (16 categories) -> 15 dummy column(s)
  'Exterior 2nd' (17 categories) -> 16 dummy column(s)
  'Exter Qual' (4 categories) -> 3 dummy column(s)
  'Exter Cond' (5 categories) -> 4 dummy column(s)
  'Foundation' (6 categories) -> 5 dummy column

---
## STEP 8: Correlation Analysis (Optional)

This shows which features are most related to your target. Higher values (positive or negative) mean stronger relationships.

In [9]:
print("=" * 60)
print("CORRELATION ANALYSIS")
print("=" * 60)

try:
    if TARGET_COLUMN not in df.columns:
        print("Warning: Target column not found. Skipping correlation analysis.")
    elif PROBLEM_TYPE == 'regression' or PROBLEM_TYPE == 'binary_classification':
        # Calculate correlations
        corr_matrix = df.corr(numeric_only=True)
        
        # FIX: Check if target is in correlation matrix
        if TARGET_COLUMN not in corr_matrix.columns:
            print("Warning: Could not compute correlations (target column has no variance).")
        else:
            correlations = corr_matrix[TARGET_COLUMN].drop(TARGET_COLUMN, errors='ignore')
            correlations = correlations.abs().sort_values(ascending=False)
            
            print("Top 10 features most related to your target:")
            print("")
            for feature, corr in correlations.head(10).items():
                bar = "*" * int(corr * 20)
                # FIX: Better handling of long feature names
                display_name = feature if len(feature) <= 40 else feature[:37] + "..."
                print(f"  {display_name:<40} {corr:.3f} {bar}")
    else:
        print("Correlation analysis is most useful for regression and binary classification.")
        print("Skipping for multiclass classification.")
except Exception as e:
    print(f"Warning: Could not compute correlations ({e}).")
    print("Continuing with data preparation.")

CORRELATION ANALYSIS
Top 10 features most related to your target:

  Overall Qual                             0.799 ***************
  Gr Liv Area                              0.707 **************
  Garage Cars                              0.648 ************
  Garage Area                              0.640 ************
  Total Bsmt SF                            0.632 ************
  1st Flr SF                               0.622 ************
  Exter Qual_TA                            0.591 ***********
  Year Built                               0.558 ***********
  Full Bath                                0.546 **********
  Year Remod/Add                           0.533 **********


---
## STEP 9: Final Validation and Formatting

This step prepares the data in the exact format required by AWS SageMaker Linear Learner:
- Target column first
- All values converted to decimal numbers
- No missing values
- Remove duplicate rows

In [10]:
print("=" * 60)
print("FINAL VALIDATION")
print("=" * 60)

# Verify target column exists
if TARGET_COLUMN not in df.columns:
    print("CRITICAL ERROR: Target column was lost during processing.")
    raise ValueError(f"Target column '{TARGET_COLUMN}' not found in processed data.")

# Check that all columns are numeric
non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric:
    print(f"ERROR: Found non-numeric columns: {non_numeric}")
    print("These columns could not be converted automatically.")
    raise ValueError("Non-numeric columns remain.")
print("All columns are numeric.")

# Handle infinite values if present
inf_count = np.isinf(df.values).sum()
if inf_count > 0:
    print(f"Warning: Found {inf_count} infinite value(s). Replacing with column median.")
    df = df.replace([np.inf, -np.inf], np.nan)
else:
    print("No infinite values.")

# Fill any remaining missing values (including those created by cleaning steps)
for col in df.columns:
    if df[col].isna().sum() > 0:
        if col == TARGET_COLUMN and PROBLEM_TYPE != 'regression':
            # For classification, drop rows with invalid target instead of filling
            invalid_rows = df[col].isna().sum()
            if invalid_rows > 0:
                print(f"Warning: {invalid_rows} rows had invalid target values (removed).")
                df = df.dropna(subset=[TARGET_COLUMN])
        else:
            # Fill features with median
            median_val = df[col].median()
            if pd.isna(median_val):
                median_val = 0  # Fallback if entire column was empty
            df.loc[:, col] = df[col].fillna(median_val)

print("Missing values handled.")

# Final check for missing values
total_missing = df.isna().sum().sum()
if total_missing > 0:
    print(f"ERROR: {total_missing} missing values remain.")
    raise ValueError("Missing values remain.")
print("No missing values.")

# Remove duplicate rows
n_duplicates = df.duplicated().sum()
if n_duplicates > 0:
    df = df.drop_duplicates()
    print(f"Removed {n_duplicates} duplicate rows.")
else:
    print("No duplicate rows.")

# Verify and re-encode target classes if necessary
if PROBLEM_TYPE in ['binary_classification', 'multiclass_classification']:
    actual_classes = sorted(df[TARGET_COLUMN].unique())
    expected_classes = list(range(len(actual_classes)))
    
    if actual_classes != expected_classes:
        print(f"")
        print(f"Note: Re-encoding target classes to ensure sequential numbering.")
        print(f"  Classes present: {actual_classes} -> Re-encoding to: {expected_classes}")
        new_mapping = {old: new for new, old in enumerate(actual_classes)}
        df[TARGET_COLUMN] = df[TARGET_COLUMN].map(new_mapping)
        
        # Update original mapping for reference
        if ORIGINAL_CLASS_MAPPING:
            inverse_original = {v: k for k, v in ORIGINAL_CLASS_MAPPING.items()}
            updated_mapping = {}
            for old_encoded, new_encoded in new_mapping.items():
                if old_encoded in inverse_original:
                    original_label = inverse_original[old_encoded]
                    updated_mapping[original_label] = new_encoded
            ORIGINAL_CLASS_MAPPING = updated_mapping
            print("  Updated class mapping.")
        
        # Check if problem type shifted (e.g., multiclass -> binary)
        if len(actual_classes) == 2 and PROBLEM_TYPE == 'multiclass_classification':
            PROBLEM_TYPE = 'binary_classification'
            print(f"  Problem type changed to: binary classification")
        elif len(actual_classes) == 1:
            print("")
            print("!" * 60)
            print("ERROR: ONLY ONE CLASS REMAINS")
            print("!" * 60)
            print("After cleaning, only one target class remains.")
            print("A classifier cannot learn from data with only one class.")
            raise ValueError("Only one target class remaining.")

# Check for float32 overflow before conversion
max_float32 = np.finfo(np.float32).max
min_float32 = np.finfo(np.float32).min
overflow_cols = []
for col in df.columns:
    col_max = df[col].max()
    col_min = df[col].min()
    if col_max > max_float32 or col_min < min_float32:
        overflow_cols.append(col)
        df[col] = df[col].clip(lower=min_float32, upper=max_float32)

if overflow_cols:
    print(f"Warning: Clipped {len(overflow_cols)} column(s) with values exceeding float32 range.")

# Convert all to float32 (required by Linear Learner)
df = df.astype(np.float32)
print("Converted all values to float32.")

# Move target column to first position
cols = [TARGET_COLUMN] + [c for c in df.columns if c != TARGET_COLUMN]
df = df[cols]
print("Moved target column to first position.")

# Calculate final dimensions
n_features = len(df.columns) - 1
n_rows = len(df)

# Track duplicates removed for summary
ROWS_DROPPED_DUPLICATES = n_duplicates

print("")
print(f"Final dataset: {n_rows:,} rows x {len(df.columns)} columns")
print(f"Features: {n_features}")

FINAL VALIDATION
All columns are numeric.
No infinite values.
Missing values handled.
No missing values.
No duplicate rows.
Converted all values to float32.
Moved target column to first position.

Final dataset: 2,930 rows x 228 columns
Features: 227


---
## STEP 10: Preparation Summary and Recommendation

This summary tells you whether your dataset is ready for SageMaker.

In [11]:
print("=" * 60)
print("PREPARATION COMPLETE")
print("=" * 60)

# Calculate total rows dropped
ROWS_DROPPED = ROWS_DROPPED_MISSING + ROWS_DROPPED_DUPLICATES

# Summarize what was cleaned
changes = []
if COLS_DROPPED_USELESS > 0:
    changes.append(f"Removed {COLS_DROPPED_USELESS} useless column(s) (IDs, constants)")
if COLS_DROPPED_MISSING > 0:
    changes.append(f"Removed {COLS_DROPPED_MISSING} column(s) with too many missing values")
if COLS_DROPPED_CARDINALITY > 0:
    changes.append(f"Removed {COLS_DROPPED_CARDINALITY} column(s) with too many text categories")
if ROWS_DROPPED > 0:
    changes.append(f"Removed {ROWS_DROPPED:,} row(s) (missing data, duplicates)")
if COLS_ENCODED > 0:
    changes.append(f"Converted {COLS_ENCODED} text column(s) to numbers")

if changes:
    print("")
    print("What was cleaned:")
    for change in changes:
        print(f"  - {change}")

print("")
print(f"Final dataset: {n_rows:,} rows x {n_features} features")
print(f"Problem type: {PROBLEM_TYPE.replace('_', ' ').title()}")

# Evaluate dataset suitability
issues = []

# Check 1: Too few rows
if n_rows < 300:
    issues.append("Very few rows remaining (less than 300)")

# Check 2: More features than rows (after encoding)
if n_features > n_rows:
    issues.append(f"More columns ({n_features}) than rows ({n_rows:,}) after encoding")

# Check 3: Lost most of the data
retention_rate = n_rows / RAW_ROWS
if retention_rate < 0.3:
    issues.append(f"More than 70% of original data was removed")

# Check 4: Class imbalance (classification only) - IMPROVED for binary and multiclass
if PROBLEM_TYPE != 'regression':
    class_distribution = df[TARGET_COLUMN].value_counts(normalize=True).sort_index()
    max_class_pct = class_distribution.max()
    min_class_pct = class_distribution.min()
    n_classes = df[TARGET_COLUMN].nunique()
    
    if PROBLEM_TYPE == 'binary_classification':
        # For binary classification, focus on minority class percentage
        if min_class_pct < 0.05:
            issues.append(f"Minority class is only {min_class_pct*100:.1f}% of data (severe imbalance)")
        elif min_class_pct < 0.15:
            issues.append(f"Minority class is only {min_class_pct*100:.1f}% of data (significant imbalance)")
        elif min_class_pct < 0.25:
            issues.append(f"Minority class is {min_class_pct*100:.1f}% of data (moderate imbalance)")
    else:
        # For multiclass, check if any single class dominates
        expected_balanced_pct = 1.0 / n_classes
        if max_class_pct > 0.95:
            issues.append(f"One category represents {max_class_pct*100:.1f}% of the data (severe imbalance)")
        elif max_class_pct > expected_balanced_pct * 4:
            issues.append(f"One category represents {max_class_pct*100:.1f}% of the data (class imbalance)")
    
    # Always show distribution for classification problems
    print("")
    print("Target class distribution:")
    for class_val, pct in class_distribution.items():
        # Look up original label if available
        original_label = class_val
        if ORIGINAL_CLASS_MAPPING:
            for orig, enc in ORIGINAL_CLASS_MAPPING.items():
                if enc == class_val:
                    original_label = orig
                    break
        bar = "*" * int(pct * 40)
        print(f"  {original_label}: {pct*100:5.1f}% {bar}")

# Display recommendation
print("")
print("=" * 60)
print("RECOMMENDATION")
print("=" * 60)

if len(issues) == 0:
    print("")
    print(">>> PROCEED <<<")
    print("")
    print("Your dataset passed all quality checks and is ready for SageMaker.")
    print("Continue to Step 11 to export your cleaned file.")
    RECOMMENDATION = "PROCEED"
elif len(issues) == 1:
    print("")
    print(">>> PROCEED WITH CAUTION <<<")
    print("")
    print("Your dataset can be used, but there is one concern:")
    print(f"  - {issues[0]}")
    print("")
    print("You may continue to Step 11, but model performance may be limited.")
    print("Consider finding a better dataset if results are poor.")
    RECOMMENDATION = "CAUTION"
else:
    print("")
    print(">>> STOP - CONSIDER A DIFFERENT DATASET <<<")
    print("")
    print("Your dataset has multiple issues that may prevent good results:")
    for issue in issues:
        print(f"  - {issue}")
    print("")
    print("You can still export and try training, but results will likely be poor.")
    print("Recommendation: Choose a different dataset or consult your instructor.")
    RECOMMENDATION = "STOP"

print("")
print("=" * 60)

PREPARATION COMPLETE

What was cleaned:
  - Removed 1 useless column(s) (IDs, constants)
  - Removed 6 column(s) with too many missing values
  - Converted 37 text column(s) to numbers

Final dataset: 2,930 rows x 227 features
Problem type: Regression

RECOMMENDATION

>>> PROCEED <<<

Your dataset passed all quality checks and is ready for SageMaker.
Continue to Step 11 to export your cleaned file.



---
## STEP 11: Export Cleaned Data

This creates the final CSV file ready for upload to AWS SageMaker.

In [12]:
# Create output filename (FIX: sanitize special characters)
base_name = os.path.splitext(FILE_NAME)[0]
# Replace spaces and special characters with underscores for S3 compatibility
safe_base_name = re.sub(r'[^\w\-]', '_', base_name)
output_file = f"{safe_base_name}_CLEANED.csv"

# Save without headers or index (required by Linear Learner)
df.to_csv(output_file, index=False, header=False)

print("=" * 60)
print("FILE EXPORTED")
print("=" * 60)
print(f"")
print(f"Saved as: {output_file}")
print(f"")
print(f"File size: {n_rows:,} rows x {len(df.columns)} columns")
print(f"")
print("Preview of first 5 rows:")
print(df.head().to_string(index=False, header=False))

FILE EXPORTED

Saved as: AmesHousing_CLEANED.csv

File size: 2,930 rows x 228 columns

Preview of first 5 rows:
215000.0 526301088.0 20.0 141.0 31770.0 0.151195 6.0 5.0 1960.0 1960.0 112.0  639.0   0.0  441.0 1080.0 1656.0   0.0 0.0 1656.0 1.0 0.0 1.0 0.0 3.0 1.0 7.0 2.0 1960.0 2.0 528.0 210.0 62.0 0.0 0.0   0.0 0.0     0.0 5.0 2010.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

---
## Troubleshooting

**"Target column not found"**
- Check the spelling of your target column name
- Column names are case-sensitive ("Price" is different from "price")
- Check for extra spaces before or after the column name
- Look at the list of available columns in the error message

**"Too many categories in text target column"**
- Your target column has text values with too many unique entries
- Classification works best with fewer than 20 categories
- Consider grouping similar categories together, or use a numeric target for regression

**"Target must have at least 2 unique values"**
- Your target column has only one value (or all values are missing)
- A model cannot learn to predict when there is nothing to distinguish
- Check that you selected the correct target column

**"Too few rows remaining"**
- Your dataset has too many missing values
- Try a different dataset with more complete data

**"More columns than rows"**
- After converting text to numbers, you have too many columns
- This happens with datasets that have many text categories
- Try a dataset with fewer text columns or more rows

**"Dataset may not work well"**
- The notebook found issues that may prevent good results
- You can still try training, but consider using a different dataset

**Questions?**
- Consult your instructor or TA for help choosing an appropriate dataset